## Imports

In [ ]:
!pip install cmake dlib
!pip install face-recognition
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import face_recognition
import smtplib
import imghdr
from email.message import EmailMessage
import ssl

## Main model for Web Cams detection
Multiple web cams can be used in this model, but we are only using 1 either the Laptop's webcam or the phone's camera

In [ ]:
video_capture = cv2.VideoCapture(0)

In [ ]:
auth_person_image = face_recognition.load_image_file("/Users/manav/Downloads/auth_person.jpg")
auth_encoding = face_recognition.face_encodings(auth_person_image)[0]

In [ ]:
known_face_encodings = [auth_encoding]
known_face_names = ["Authorised"]

face_locations = []
face_encodings = []
face_names = []
check_intruder = 0
count_occ = 0
process_this_frame = True

In [ ]:
while True:
  ret, frame = video_capture.read() # to get frame

  if process_this_frame:
    small_frame = cv2.resize(frame, (0,0), fx=0.25, fy=0.25) # reducing the size of the frame for faster processing
    
    rgb_small_frame = small_frame[:, :, ::-1] # normalising the image
    face_locations = face_recognition.face_locations(rgb_small_frame) # face locations
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations) # and face encodings 

    face_names = []
    for face_encoding in face_encodings:
      matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
      name = "Unknown"
      print(matches)
      
      face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
      best_match_index = np.argmin(face_distances)
      # print(face_distances)
      # print(best_match_index)
      if matches[best_match_index]:
        print(matches[best_match_index])
        name = known_face_names[best_match_index]
        check_intruder = 0
        count_occ = 0

      face_names.append(name)
      if name == "Unknown":
        check_intruder = 1 
  

  if check_intruder == 1:
    count_occ += 1
  if count_occ > 100:
    return_value, image = video_capture.read()
    cv2.imwrite('/Users/manav/Downloads/ImageRecognition/opnecv.png', image)
   
    smtp_server = "smtp.gmail.com"
    port = 587  # For starttls
    sender_email = "kate07seven@gmail.com"
    password = "nnvnxdbeusfoxcxf"

    # Create a secure SSL context
    context = ssl.create_default_context()
    server = smtplib.SMTP(smtp_server, port)
    server.starttls(context=context)  # Secure the connection
    server.login(sender_email, password)
    new_msg = EmailMessage()
    new_msg['Subject'] = "Intruder Alert"
    new_msg['From'] = "kate47@gmail.com"
    new_msg['To'] = "gyanviu@gmail.com"
    new_msg.set_content("Intruder picture : ")
    with open('/Users/manav/Downloads/ImageRecognition/opnecv.png', 'rb') as f:
      image_data = f.read()
      image_type = imghdr.what(f.name)
      image_name = f.name
    new_msg.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)
    server.send_message(new_msg)
    break
    
  # To avoid processing all the frames
  process_this_frame = not process_this_frame

  # Display results
  for(top, right, bottom, left), name in zip(face_locations, face_names):
    top *= 4
    right *= 4
    bottom *= 4
    left *= 4

    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
  
  cv2.imshow("webcam", frame)

  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

video_capture.release()
cv2.destroyAllWindows()


This part to be executed in terminal